In [23]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter


# 1. Load Document

In [24]:
loader = PyPDFLoader("docs/Texto-aprobado-Consejo-Constitucional_06.10.23.pdf")
pages = loader.load()

In [25]:
len(pages)

107

In [26]:
len(pages[0].page_content)

2912

In [27]:
print(pages[0].page_content[0:1000])

 
1 
  
TEXTO APROBADO POR EL CONSEJO CONSTITUCIONAL  
 
 
CAPÍTULO I  
 
FUNDAMENTOS DEL ORDEN CONSTITUCIONAL  
 
Artículo 1  
 
1. La dignidad humana es inviolable y la base del derecho y la justicia. Las personas nacen 
libres e iguales en dignidad y derechos. Su respeto y garantía es el primer deber de la 
comunidad política y de su forma jurídica de organización.  
 
2. La familia es el núcleo fundamental de la sociedad. Es deber del Estado y la sociedad dar 
protección a las familias y propender a su for talecimiento.  
 
3. Las agrupaciones que libremente surjan entre las personas gozarán de la adecuada 
autonomía para cumplir sus fines específicos que no sean contrarios a la Constitución. El 
Estado respetará los efectos de este reconocimiento . 
 
Artículo 2  
 
1. El Estado de Chile es social y democrático de derecho, que reconoce  derechos y libertades 
fundamentales, deberes constitucionales, y promueve el desarrollo progresivo de los 
derechos sociales, con sujeción al pri

In [28]:
pages[0].metadata

{'source': 'docs/Texto-aprobado-Consejo-Constitucional_06.10.23.pdf',
 'page': 0}

# 2. Split Document (1500, 300)

In [29]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "\. ", " ", ""],
    chunk_size=1500,
    chunk_overlap=300,
    length_function=len
)

In [33]:
docs = text_splitter.split_documents(pages)

In [34]:
len(docs)

308

In [35]:
print(docs[0].page_content), print(docs[1].page_content)

1 
  
TEXTO APROBADO POR EL CONSEJO CONSTITUCIONAL  
 
 
CAPÍTULO I  
 
FUNDAMENTOS DEL ORDEN CONSTITUCIONAL  
 
Artículo 1  
 
1. La dignidad humana es inviolable y la base del derecho y la justicia. Las personas nacen 
libres e iguales en dignidad y derechos. Su respeto y garantía es el primer deber de la 
comunidad política y de su forma jurídica de organización.  
 
2. La familia es el núcleo fundamental de la sociedad. Es deber del Estado y la sociedad dar 
protección a las familias y propender a su for talecimiento.  
 
3. Las agrupaciones que libremente surjan entre las personas gozarán de la adecuada 
autonomía para cumplir sus fines específicos que no sean contrarios a la Constitución. El 
Estado respetará los efectos de este reconocimiento . 
 
Artículo 2  
 
1. El Estado de Chile es social y democrático de derecho, que reconoce  derechos y libertades 
fundamentales, deberes constitucionales, y promueve el desarrollo progresivo de los 
derechos sociales, con sujeción al princ

(None, None)

# 3. Create Embeddings

In [36]:
from langchain.embeddings import BedrockEmbeddings 
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1")

In [37]:
bedrock_embeddings.embed_query(text="Hola Mundo")[:3]

[0.51953125, 0.44921875, 0.32421875]

In [38]:
from langchain.vectorstores import Chroma

In [41]:
persist_directory = 'docs/chroma/'
!rm -rf ./docs/chroma  # remove old database files if any

In [42]:
%%time
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=bedrock_embeddings,
    persist_directory=persist_directory
)
print(vectordb._collection.count())

308
CPU times: user 2.14 s, sys: 365 ms, total: 2.51 s
Wall time: 1min 33s


In [43]:
%%time
question = "como la constitución protege el derecho a la propiedad?"
responses = vectordb.similarity_search(question,k=3)

CPU times: user 7.77 ms, sys: 2.21 ms, total: 9.98 ms
Wall time: 848 ms


In [44]:
responses

[Document(page_content='16 \n  \nSolo en virtud de una ley de quorum  calificado, y siempre que no signifique tal \ndiscriminación, se podrán autorizar determinados beneficios directos o indirectos en favor de \nalgún sector, actividad o zona geográfica, o establecer gravámenes especiales que afecten a \nuno u otras. En el caso de las franquicias o beneficios  indirectos, la estimación del costo de \nestos deberá incluirse anualmente en la Ley de Presupuestos.  \n \n34. La libertad para adquirir el dominio de toda clase de bienes, excepto aquellos que la \nnaturaleza ha hecho comunes a todas las personas o que deban per tenecer a la Nación toda y \nla ley lo declare así. Lo anterior es sin perjuicio de lo prescrito en otros preceptos de esta \nConstitución.  \n \nUna ley de quorum  calificado, cuando así lo exija el interés nacional, podrá establecer \nlimitaciones o requisitos para  la adquisición del dominio de algunos bienes.  \n \n35. El derecho de propiedad en sus diversas especie

In [45]:
vectordb.persist()

In [46]:
print(vectordb._collection.count())

308


In [47]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)
docs_mmr

[Document(page_content='16 \n  \nSolo en virtud de una ley de quorum  calificado, y siempre que no signifique tal \ndiscriminación, se podrán autorizar determinados beneficios directos o indirectos en favor de \nalgún sector, actividad o zona geográfica, o establecer gravámenes especiales que afecten a \nuno u otras. En el caso de las franquicias o beneficios  indirectos, la estimación del costo de \nestos deberá incluirse anualmente en la Ley de Presupuestos.  \n \n34. La libertad para adquirir el dominio de toda clase de bienes, excepto aquellos que la \nnaturaleza ha hecho comunes a todas las personas o que deban per tenecer a la Nación toda y \nla ley lo declare así. Lo anterior es sin perjuicio de lo prescrito en otros preceptos de esta \nConstitución.  \n \nUna ley de quorum  calificado, cuando así lo exija el interés nacional, podrá establecer \nlimitaciones o requisitos para  la adquisición del dominio de algunos bienes.  \n \n35. El derecho de propiedad en sus diversas especie